## Github link: https://github.com/Dante-1802/Assignment.git

## Bài 1:

Theo định nghĩa của bài toán, ta có:
* Hàm mất mát: $L(o,y) = -\sum_{c=1}^{C}\mathbb{I}(y=c)\log P(c|x)$
* Hàm Softmax: $P(c|x) = p_c = \frac{\exp(o_c)}{\sum_{c^{\prime}=1}^{C}\exp(o_{c^{\prime}})}$
* Đầu ra lớp cuối: $o = z_L$ (không có hàm kích hoạt ở lớp cuối)
* Tính toán tại lớp $l$: $z_{l}=W_{l}o_{l-1}+b_{l}$ và $o_{l}=\sigma_{l}(z_{l})$
* Ta định nghĩa $y_{one\_hot}$ là véc-tơ one-hot của nhãn $y$, với $y_c = \mathbb{I}(y=c)$.

---

### 1. Đạo hàm của hàm mất mát theo đầu ra $o$

Đạo hàm của hàm mất mát Cross-Entropy đối với đầu vào $o$ của hàm Softmax là:

$$
\frac{\partial L}{\partial o_a} = p_a - y_a = P(a|x) - \mathbb{I}(y=a)
$$

Ở dạng véc-tơ, ta có $\delta_L$ (gradient tại đầu ra của lớp cuối cùng $L$):

$$
\delta_L = \nabla_o L = p - y_{one\_hot}
$$

Trong đó, $p$ là véc-tơ xác suất đầu ra từ softmax, và $y_{one\_hot}$ là véc-tơ one-hot của nhãn thực tế.

---

### 2. Đạo hàm của hàm mất mát theo đầu ra $o_l$

Chúng ta sẽ sử dụng quy tắc chuỗi (chain rule). Ta cần tính $\delta_l = \nabla_{o_l} L$ dựa trên $\delta_{l+1} = \nabla_{o_{l+1}} L$.

Trước tiên, ta định nghĩa gradient trung gian $\Delta_{l+1} = \nabla_{z_{l+1}} L$.
Sử dụng quy tắc chuỗi:
$$
\Delta_{l+1} = \nabla_{z_{l+1}} L = \nabla_{o_{l+1}} L \odot \frac{\partial o_{l+1}}{\partial z_{l+1}} = \delta_{l+1} \odot \sigma'_{l+1}(z_{l+1})
$$ 
(với $\odot$ là phép nhân theo từng phần tử).

Bây giờ, ta lan truyền gradient từ $z_{l+1}$ về $o_l$:
$$
\frac{\partial L}{\partial o_{l,a}} = \sum_{i=1}^{d_{l+1}} \frac{\partial L}{\partial z_{l+1, i}} \cdot \frac{\partial z_{l+1, i}}{\partial o_{l,a}}
$$
Vì $z_{l+1} = W_{l+1}o_l + b_{l+1}$, ta có $\frac{\partial z_{l+1, i}}{\partial o_{l,a}} = W_{l+1, i, a}$.

$$
\frac{\partial L}{\partial o_{l,a}} = \sum_{i=1}^{d_{l+1}} (\Delta_{l+1})_i \cdot W_{l+1, i, a}
$$
Đây chính là phép nhân ma trận-véc-tơ $(W_{l+1})^T \Delta_{l+1}$.

Ở dạng véc-tơ:
$$
\delta_l = \nabla_{o_l} L = (W_{l+1})^T \Delta_{l+1} = (W_{l+1})^T (\delta_{l+1} \odot \sigma'_{l+1}(z_{l+1}))
$$

---

### 3. Đạo hàm của hàm mất mát theo trọng số $W_l$ và $b_l$

Ta tiếp tục sử dụng gradient trung gian $\Delta_l = \nabla_{z_l} L$.
$$
\Delta_l = \nabla_{z_l} L = \delta_l \odot \sigma'_l(z_l)
$$
*(Lưu ý: Đối với lớp cuối $L$, vì $o_L = z_L$ nên $\sigma'_L = 1$. Do đó, $\Delta_L = \delta_L = p - y_{one\_hot}$)*

**Đối với độ lệch $b_l$:**
$$
\frac{\partial L}{\partial b_{l,a}} = \frac{\partial L}{\partial z_{l,a}} \cdot \frac{\partial z_{l,a}}{\partial b_{l,a}} = (\Delta_l)_a \cdot 1
$$
Ở dạng véc-tơ:
$$
\nabla_{b_l} L = \Delta_l
$$

**Đối với trọng số $W_l$:**
$$
\frac{\partial L}{\partial W_{l,a,b}} = \frac{\partial L}{\partial z_{l,a}} \cdot \frac{\partial z_{l,a}}{\partial W_{l,a,b}}
$$
Vì $z_{l,a} = \sum_j W_{l,a,j} o_{l-1, j} + b_{l,a}$, ta có $\frac{\partial z_{l,a}}{\partial W_{l,a,b}} = o_{l-1, b}$.
$$
\frac{\partial L}{\partial W_{l,a,b}} = (\Delta_l)_a \cdot o_{l-1, b}
$$
Đây chính là tích ngoài (outer product) của véc-tơ $\Delta_l$ và $o_{l-1}$.

Ở dạng ma trận:
$$
\nabla_{W_l} L = \Delta_l (o_{l-1})^T
$$

---

### 4. Công thức cập nhật tham số (SGD)

Sử dụng thuật toán SGD (với kích thước lô bằng 1) và tốc độ học $n$:

$$
W_l \leftarrow W_l - n \cdot \nabla_{W_l} L = W_l - n \cdot \Delta_l (o_{l-1})^T
$$
$$
b_l \leftarrow b_l - n \cdot \nabla_{b_l} L = b_l - n \cdot \Delta_l
$$

## Bài 2:

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Import các mô hình
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
# Sử dụng hạt giống ngẫu nhiên cố định
RANDOM_SEED = 42

# Tải dữ liệu
digits = load_digits()
X, y = digits.data, digits.target

# Chia 60% train, 20% validation, 20% test
# Tách test set ra trước (20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED
)

# Tách train và validation từ 80% còn lại (0.25 * 0.8 = 0.2)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=RANDOM_SEED
)

print(f"Kích thước tập Train: {X_train.shape[0]} (60%)")
print(f"Kích thước tập Validation: {X_val.shape[0]} (20%)")
print(f"Kích thước tập Test: {X_test.shape[0]} (20%)")

# --- Tiền xử lý: Scaling ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

Kích thước tập Train: 1077 (60%)
Kích thước tập Validation: 360 (20%)
Kích thước tập Test: 360 (20%)


In [3]:
# Nơi lưu trữ các mô hình tốt nhất và siêu tham số
best_models = {}
results = {}

print("\n--- Bắt đầu Huấn luyện & Tối ưu ---")

## 1. Logistic Regression
print("Đang tối ưu Logistic Regression...")
best_acc_lr = 0
best_C_lr = None
solver = 'lbfgs'
for C in [0.01, 0.1, 1, 10]:
    model_lr = LogisticRegression(C=C, solver=solver, max_iter=2000, random_state=RANDOM_SEED)
    model_lr.fit(X_train_scaled, y_train)
    val_acc = model_lr.score(X_val_scaled, y_val)
    
    if val_acc > best_acc_lr:
        best_acc_lr = val_acc
        best_C_lr = C
        best_models['Logistic Regression'] = model_lr

print(f"LR: Siêu tham số tốt nhất: C={best_C_lr}, Solver={solver}")


--- Bắt đầu Huấn luyện & Tối ưu ---
Đang tối ưu Logistic Regression...
LR: Siêu tham số tốt nhất: C=0.1, Solver=lbfgs


In [4]:
## 2. Decision Tree
print("Đang tối ưu Decision Tree...")
best_acc_dt = 0
best_depth_dt = None
for depth in [5, 10, 15, None]:
    model_dt = DecisionTreeClassifier(max_depth=depth, random_state=RANDOM_SEED)
    model_dt.fit(X_train, y_train)
    val_acc = model_dt.score(X_val, y_val)
    
    if val_acc > best_acc_dt:
        best_acc_dt = val_acc
        best_depth_dt = depth
        best_models['Decision Tree'] = model_dt

print(f"DT: Siêu tham số tốt nhất: max_depth={best_depth_dt}")

Đang tối ưu Decision Tree...
DT: Siêu tham số tốt nhất: max_depth=10


In [5]:
## 3. K-Nearest Neighbors (KNN)
print("Đang tối ưu KNN...")
best_acc_knn = 0
best_k_knn = None
for k in [1, 3, 5, 7, 9]:
    model_knn = KNeighborsClassifier(n_neighbors=k)
    model_knn.fit(X_train_scaled, y_train)
    val_acc = model_knn.score(X_val_scaled, y_val)
    
    if val_acc > best_acc_knn:
        best_acc_knn = val_acc
        best_k_knn = k
        best_models['K-Nearest Neighbors'] = model_knn

print(f"KNN: Siêu tham số tốt nhất: n_neighbors={best_k_knn}")

Đang tối ưu KNN...
KNN: Siêu tham số tốt nhất: n_neighbors=1


In [6]:
## 4. Artificial Neural Network (MLPClassifier)
print("Đang tối ưu ANN (MLP)...")
best_acc_ann = 0
best_params_ann = {}
for hidden_layer_sizes in [(50,), (100,), (50, 25)]:
    for alpha in [0.0001, 0.001]:
        model_ann = MLPClassifier(
            hidden_layer_sizes=hidden_layer_sizes,
            alpha=alpha,
            max_iter=1000,
            random_state=RANDOM_SEED,
            early_stopping=True
        )
        model_ann.fit(X_train_scaled, y_train)
        val_acc = model_ann.score(X_val_scaled, y_val)
        
        if val_acc > best_acc_ann:
            best_acc_ann = val_acc
            best_params_ann = {'hidden_layer_sizes': hidden_layer_sizes, 'alpha': alpha}
            best_models['ANN (MLP)'] = model_ann

print(f"ANN: Siêu tham số tốt nhất: {best_params_ann}")

Đang tối ưu ANN (MLP)...
ANN: Siêu tham số tốt nhất: {'hidden_layer_sizes': (100,), 'alpha': 0.0001}


In [7]:
print("\n--- Đánh giá trên tập Test ---")

final_results_table = []

for model_name, model in best_models.items():
    if model_name == 'Decision Tree':
        X_test_final = X_test
        params = f"max_depth={model.max_depth}"
    elif model_name == 'Logistic Regression':
        X_test_final = X_test_scaled
        params = f"C={model.C}, solver={model.solver}"
    elif model_name == 'K-Nearest Neighbors':
        X_test_final = X_test_scaled
        params = f"n_neighbors={model.n_neighbors}"
    elif model_name == 'ANN (MLP)':
        X_test_final = X_test_scaled
        params = f"hidden_layer_sizes={model.hidden_layer_sizes}, alpha={model.alpha}"
    
    # Dự đoán
    y_pred = model.predict(X_test_final)
    
    # Tính toán
    test_accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Chuẩn bị dữ liệu cho bảng
    row = {
        "Mô hình": model_name,
        "Siêu tham số tốt nhất": params,
        "Accuracy (Test)": f"{test_accuracy:.4f}"
    }
    # Thêm recall cho từng lớp 
    for c in range(10):
        row[f"Recall Lớp {c}"] = f"{report[str(c)]['recall']:.4f}"
        
    final_results_table.append(row)
    
    print(f"\n{model_name}:")
    print(f"  Siêu tham số: {params}")
    print(f"  Độ chính xác (Test): {test_accuracy:.4f}")


--- Đánh giá trên tập Test ---

Logistic Regression:
  Siêu tham số: C=0.1, solver=lbfgs
  Độ chính xác (Test): 0.9694

Decision Tree:
  Siêu tham số: max_depth=10
  Độ chính xác (Test): 0.8528

K-Nearest Neighbors:
  Siêu tham số: n_neighbors=1
  Độ chính xác (Test): 0.9639

ANN (MLP):
  Siêu tham số: hidden_layer_sizes=(100,), alpha=0.0001
  Độ chính xác (Test): 0.9583


In [8]:
# Sử dụng Pandas để hiển thị bảng kết quả 
df_results = pd.DataFrame(final_results_table)

# Sắp xếp các cột theo yêu cầu (11 cột)
columns_order = ["Mô hình", "Siêu tham số tốt nhất", "Accuracy (Test)"] + [f"Recall Lớp {c}" for c in range(10)]
df_results = df_results[columns_order]

print("\n\n--- Bảng tổng hợp kết quả ---")
print(df_results.to_markdown(index=False))



--- Bảng tổng hợp kết quả ---
| Mô hình             | Siêu tham số tốt nhất                   |   Accuracy (Test) |   Recall Lớp 0 |   Recall Lớp 1 |   Recall Lớp 2 |   Recall Lớp 3 |   Recall Lớp 4 |   Recall Lớp 5 |   Recall Lớp 6 |   Recall Lớp 7 |   Recall Lớp 8 |   Recall Lớp 9 |
|:--------------------|:----------------------------------------|------------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|
| Logistic Regression | C=0.1, solver=lbfgs                     |            0.9694 |         1      |         0.9643 |         1      |         0.9412 |         1      |         0.9362 |         0.9714 |         0.9706 |         0.9667 |          0.95  |
| Decision Tree       | max_depth=10                            |            0.8528 |         0.9091 |         0.7143 |         0.8788 |         0.8529 |         0.8478 |         0.8936 |         0.94